The New York Neighborhood Recommender

I. Purpose

Just imagine finding a well payed job in New York. Finally having this grand and diverse city as yo
ur new home. As many people found out, it can also be tricky to go there. What if you feel strange? Or you find out you don't like your Neighborhood? This App will help you getting along by finding the right Neighborhood to move to based on where you live now. So you won't feel alone and strange. For anyone renting apartments it can be profitable as well. People find the right place to call home faster and are more happy, thus they will stay longer. Plus they will recommend you.

II. Data Acquisition

We will use data based on the Foursquare API. With it we can gather information on all revenues in a given Neighborhood of New Yor. We will also gather the same information for the place you are currently living in.

Lets start! First with the New York Data. We downloaded a dataset with all Neighborhoods and geolocations.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.shape

(306, 3)

Now get the hottest venues.

In [3]:
CLIENT_ID = '3A1JPR4E42ITJEPNWTDJ0AYIDDLM4ULERLLRFAUKHXDQIM3L' # your Foursquare ID
CLIENT_SECRET = 'V0ZROHHBYCLH4A2ORB4Y1EIIQNNK3OFHBV4L0ZY1WXTHMGSZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 5 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

In [4]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
        
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #print(requests.get(url).json())
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',                   
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [16]:
newyork_venues.shape

(1474, 7)

In [17]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()

print(newyork_grouped['Neighborhood'].shape)
#remove neighborhoods for which we didn't load any data befor
#neighborhoods = neighborhoods.reset_index().drop([301, 302, 303, 304, 305])
print(neighborhoods['Neighborhood'].shape)

(301,)
(301,)


In [18]:
#print(neighborhoods.sort_values(['Neighborhood']))
print(newyork_grouped.set_index('Neighborhood').join(neighborhoods.set_index('Neighborhood')).shape)




(305, 250)


In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Donut Shop,Pizza Place,Fast Food Restaurant,Dessert Shop,Martial Arts Dojo,Eastern European Restaurant,Filipino Restaurant,Field,Farmers Market,Farm
1,Annadale,Sports Bar,Train Station,Pet Store,Diner,Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space
2,Arden Heights,Pizza Place,Pharmacy,Coffee Shop,Playground,Cycle Studio,Cupcake Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market
3,Arlington,Scenic Lookout,Grocery Store,Boat or Ferry,American Restaurant,Bus Stop,Falafel Restaurant,Electronics Store,Empanada Restaurant,Event Space,Factory
4,Arrochar,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant,Eastern European Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm


Now the data for your location. I chose my home as an example. Later in the analysis we can see which New York Neighborhoods are similar bases on the data collected.

In [36]:
home_name = ['Eimsbüttel']
home_lat = ['53.555464']
home_lon = ['9.811365']

home_venues = getNearbyVenues(names=home_name,
                                   latitudes=home_lat,
                                   longitudes=home_lon
                                  )
# one hot encoding with the same columns
home_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
home_onehot['Neighborhood'] = home_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [home_onehot.columns[-1]] + list(home_onehot.columns[:-1])
home_onehot = home_onehot[fixed_columns]

home_grouped = home_onehot.groupby('Neighborhood').mean().reset_index()

#print(home_grouped.head())

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
home_venues_sorted = pd.DataFrame(columns=columns)
home_venues_sorted['Neighborhood'] = home_grouped['Neighborhood']

for ind in np.arange(home_grouped.shape[0]):
    home_venues_sorted.iloc[ind, 1:] = return_most_common_venues(home_grouped.iloc[ind, :], num_top_venues)

home_grouped_clustering = home_grouped.drop('Neighborhood', 1)

print("This is what my Neighborhood looks like: ")


This is what my Neighborhood looks like: 


In [35]:
home_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Eimsbüttel,Ice Cream Shop,Pharmacy,Sandwich Place,Dessert Shop,Donut Shop,Fast Food Restaurant,Field,Farmers Market,Farm,Eastern European Restaurant


In [37]:
# set number of clusters
kclusters = 40

newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)


# check cluster labels generated for each row in the dataframe

newyork_merged = neighborhoods


# add clustering labels
newyork_merged['Cluster Labels'] = kmeans.labels_


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

recommended = kmeans.predict(home_grouped.drop('Neighborhood', 1))
print()
print("We recommend the following cluster "+ str(recommended)+" . It looks like this:")



We recommend the following cluster [9] . It looks like this:


In [47]:
cluster = newyork_merged.loc[newyork_merged['Cluster Labels'] == 9, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]
cluster



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Williamsbridge,Caribbean Restaurant,Soup Place,Bar,Nightclub,Women's Store,Egyptian Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market
33,Morris Park,Deli / Bodega,Pizza Place,Italian Restaurant,Burger Joint,Eastern European Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
67,Red Hook,Bakery,Wine Shop,New American Restaurant,American Restaurant,Sandwich Place,Factory,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space
130,Woodside,Thai Restaurant,Bakery,Fast Food Restaurant,Pub,Latin American Restaurant,Event Space,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant
136,Kew Gardens,Pizza Place,Indian Restaurant,South American Restaurant,Juice Bar,Trail,Women's Store,Event Space,Egyptian Restaurant,Electronics Store,Empanada Restaurant
162,Queens Village,Mobile Phone Shop,Fast Food Restaurant,Sandwich Place,Discount Store,Martial Arts Dojo,Field,Farmers Market,Farm,Eastern European Restaurant,Falafel Restaurant
185,Hillcrest,Ice Cream Shop,Mobile Phone Shop,College Basketball Court,College Academic Building,Dim Sum Restaurant,Farm,Empanada Restaurant,Event Space,Factory,Falafel Restaurant
220,Sunnyside,Pizza Place,Market,Bakery,Spa,Peruvian Restaurant,American Restaurant,Seafood Restaurant,Grocery Store,Gym,Electronics Store
238,Butler Manor,Baseball Field,Convenience Store,Pool,Bus Stop,Egyptian Restaurant,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market
241,Arden Heights,Pizza Place,Pharmacy,Coffee Shop,Playground,Cycle Studio,Cupcake Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market


In [40]:
address = 'New York'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))

/Users/dominik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Manhattan are 40.7308619, -73.9871558.


Finally here you can inspect all clusters by yourself.

In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><iframe src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxoZWFkPiAgICAKICAgIDxtZXRhIGh0dHAtZXF1aXY9ImNvbnRlbnQtdHlwZSIgY29udGVudD0idGV4dC9odG1sOyBjaGFyc2V0PVVURi04IiAvPgogICAgPHNjcmlwdD5MX1BSRUZFUl9DQU5WQVMgPSBmYWxzZTsgTF9OT19UT1VDSCA9IGZhbHNlOyBMX0RJU0FCTEVfM0QgPSBmYWxzZTs8L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmpzIj48L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2FqYXguZ29vZ2xlYXBpcy5jb20vYWpheC9saWJzL2pxdWVyeS8xLjExLjEvanF1ZXJ5Lm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvanMvYm9vdHN0cmFwLm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvTGVhZmxldC5hd2Vzb21lLW1hcmtlcnMvMi4wLjIvbGVhZmxldC5hd2Vzb21lLW1hcmtlcnMuanMiPjwvc2NyaXB0PgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL21heGNkbi5ib290c3RyYXBjZG4uY29tL2Jvb3RzdHJhcC8zLjIuMC9jc3MvYm9vdHN0cmFwLm1pbi5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvY3NzL2Jvb3RzdHJhcC10aGVtZS5taW4uY3NzIi8+CiAgICA8bGluayByZWw9InN0eWxlc2hlZXQiIGhyZWY9Imh0dHBzOi8vbWF4Y2RuLmJvb3RzdHJhcGNkbi5jb20vZm9udC1hd2Vzb21lLzQuNi4zL2Nzcy9mb250LWF3ZXNvbWUubWluLmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy9MZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy8yLjAuMi9sZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9yYXdnaXQuY29tL3B5dGhvbi12aXN1YWxpemF0aW9uL2ZvbGl1bS9tYXN0ZXIvZm9saXVtL3RlbXBsYXRlcy9sZWFmbGV0LmF3ZXNvbWUucm90YXRlLmNzcyIvPgogICAgPHN0eWxlPmh0bWwsIGJvZHkge3dpZHRoOiAxMDAlO2hlaWdodDogMTAwJTttYXJnaW46IDA7cGFkZGluZzogMDt9PC9zdHlsZT4KICAgIDxzdHlsZT4jbWFwIHtwb3NpdGlvbjphYnNvbHV0ZTt0b3A6MDtib3R0b206MDtyaWdodDowO2xlZnQ6MDt9PC9zdHlsZT4KICAgIAogICAgICAgICAgICA8c3R5bGU+ICNtYXBfZDhjMWY4MTEyM2VhNDgwNmE3ZDM3ZTcwNWI0OWRiMDAgewogICAgICAgICAgICAgICAgcG9zaXRpb24gOiByZWxhdGl2ZTsKICAgICAgICAgICAgICAgIHdpZHRoIDogMTAwLjAlOwogICAgICAgICAgICAgICAgaGVpZ2h0OiAxMDAuMCU7CiAgICAgICAgICAgICAgICBsZWZ0OiAwLjAlOwogICAgICAgICAgICAgICAgdG9wOiAwLjAlOwogICAgICAgICAgICAgICAgfQogICAgICAgICAgICA8L3N0eWxlPgogICAgICAgIAo8L2hlYWQ+Cjxib2R5PiAgICAKICAgIAogICAgICAgICAgICA8ZGl2IGNsYXNzPSJmb2xpdW0tbWFwIiBpZD0ibWFwX2Q4YzFmODExMjNlYTQ4MDZhN2QzN2U3MDViNDlkYjAwIiA+PC9kaXY+CiAgICAgICAgCjwvYm9keT4KPHNjcmlwdD4gICAgCiAgICAKCiAgICAgICAgICAgIAogICAgICAgICAgICAgICAgdmFyIGJvdW5kcyA9IG51bGw7CiAgICAgICAgICAgIAoKICAgICAgICAgICAgdmFyIG1hcF9kOGMxZjgxMTIzZWE0ODA2YTdkMzdlNzA1YjQ5ZGIwMCA9IEwubWFwKAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgJ21hcF9kOGMxZjgxMTIzZWE0ODA2YTdkMzdlNzA1YjQ5ZGIwMCcsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB7Y2VudGVyOiBbNDAuNzMwODYxOSwtNzMuOTg3MTU1OF0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB6b29tOiAxMSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIG1heEJvdW5kczogYm91bmRzLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgbGF5ZXJzOiBbXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHdvcmxkQ29weUp1bXA6IGZhbHNlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgY3JzOiBMLkNSUy5FUFNHMzg1NwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9KTsKICAgICAgICAgICAgCiAgICAgICAgCiAgICAKICAgICAgICAgICAgdmFyIHRpbGVfbGF5ZXJfZGZjOTk2NTlmYzI5NDQyNzk5NDZlZDI1ZmM2Y2JmZTggPSBMLnRpbGVMYXllcigKICAgICAgICAgICAgICAgICdodHRwczovL3tzfS50aWxlLm9wZW5zdHJlZXRtYXAub3JnL3t6fS97eH0ve3l9LnBuZycsCiAgICAgICAgICAgICAgICB7CiAgImF0dHJpYnV0aW9uIjogbnVsbCwKICAiZGV0ZWN0UmV0aW5hIjogZmFsc2UsCiAgIm1heFpvb20iOiAxOCwKICAibWluWm9vbSI6IDEsCiAgIm5vV3JhcCI6IGZhbHNlLAogICJzdWJkb21haW5zIjogImFiYyIKfQogICAgICAgICAgICAgICAgKS5hZGRUbyhtYXBfZDhjMWY4MTEyM2VhNDgwNmE3ZDM3ZTcwNWI0OWRiMDApOwogICAgICAgIAogICAgCiAgICAgICAgICAgIHZhciBjaXJjbGVfbWFya2VyXzBmZjEzMTMzOTM1ZTQ0MjhhNzk3Yzk2ZTQ2MWE1ODY5ID0gTC5jaXJjbGVNYXJrZXIoCiAgICAgICAgICA